In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Deal with housePrice data

In [61]:
housePrice = pd.read_csv('~/Downloads/County.csv')

In [62]:
housePrice = housePrice.drop(['RegionID', 'RegionType', 'StateName', 'SizeRank'], axis=1)
# drop rows with too many NAs
housePrice = housePrice.dropna(thresh=housePrice.shape[1]-24)

In [63]:
firstDateCol = 5
dates = housePrice.iloc[:, firstDateCol:]
otherinfo = housePrice.iloc[:, :firstDateCol]

# fill na with previous or next value along the row
dates = dates.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
housePrice = pd.concat([otherinfo, dates], axis=1)

In [64]:
# convert date columns into rows
housePrice = housePrice.melt(id_vars=["RegionName", "State", "Metro", "StateCodeFIPS", "MunicipalCodeFIPS"], 
            var_name="Date", 
            value_name="ZHVI")

In [65]:
# create ZHVI_prev column
housePrice = housePrice.sort_values(['State', 'RegionName', 'Date'])
housePrice['ZHVI_prev'] = np.roll(housePrice['ZHVI'], 1)
housePrice = housePrice[housePrice.Date != '2000-01-31']

In [66]:
#housePrice.to_csv('/Users/guannanliu/Documents/orie5741/Orie-4741-Project/data/housePrice.csv', index = False, header=True)

Deal with unemployment data

In [67]:
unemp = pd.read_excel('~/Downloads/Unemployment.xls')

In [68]:
unemp[['RegionName', 'State']] = unemp['Region Name'].str.split(", ", 1, expand=True)
unemp = unemp.drop(['Series ID', 'Region Name'], axis=1)

In [69]:
# change column order
cols = unemp.columns.tolist()
cols = cols[-2:] + cols[:-2]
unemp = unemp[cols]

In [70]:
firstDateCol = 3
dates = unemp.iloc[:, firstDateCol:]
otherinfo = unemp.iloc[:, :firstDateCol]

# fill na with previous or next value along the row
dates = dates.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
unemp = pd.concat([otherinfo, dates], axis=1)

In [71]:
unemp = unemp.melt(id_vars=['RegionName', 'State', 'Region Code'],
                    var_name='Date',
                    value_name='UnemploymentRate')

In [72]:
unemp = unemp.sort_values(['State', 'RegionName'])

In [73]:
#unemp.to_csv('/Users/guannanliu/Documents/orie5741/Orie-4741-Project/data/Unemployment.csv', index = False, header=True)

Hospital data

In [74]:
hospital = pd.read_csv('~/Downloads/Hospital.csv')

In [75]:
hospital = hospital[['State', 'County Name', 'Hospital overall rating']]
hospital = hospital[hospital['Hospital overall rating'] != 'Not Available']
hospital = hospital.dropna()
hospital['Hospital overall rating'] = hospital['Hospital overall rating'].astype(int)

In [76]:
num_hospital = hospital.groupby(['State', 'County Name'])['Hospital overall rating'].count().reset_index(name="#hospital")
avg_rating = hospital.groupby(['State', 'County Name'])['Hospital overall rating'].mean().reset_index(name="hospital_rating")

hospital = pd.concat([num_hospital, avg_rating.hospital_rating], axis=1)
hospital

,State,County Name,#hospital,hospital_rating
0,AK,ANCHORAGE,3,2.666667
1,AK,BETHEL,1,3.000000
2,AK,FAIRBANKS NORTH STAR,1,3.000000
3,AK,JUNEAU,1,3.000000
4,AK,KETCHIKAN GATEWAY,1,3.000000
...,...,...,...,...
1897,WY,SHERIDAN,1,3.000000
1898,WY,SWEETWATER,1,2.000000
1899,WY,TETON,1,4.000000
1900,WY,UINTA,1,3.000000


In [59]:
#hospital.to_csv('/Users/guannanliu/Documents/orie5741/Orie-4741-Project/data/Hospital.csv', index=False, header=True)